In [15]:
import qdmpy
import qdmpy.pl
import qdmpy.plot
import qdmpy.field
import qdmpy.source

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

In [16]:
# '%matplotlib widget' allows interactive plots in jupyterlab
# use ''%matplotlib inline' to allow export to pdf
# %matplotlib widget

In [17]:
exec(open('../TEST_DATA_PATH.py').read())

### Load raw data

In [18]:
options_dict = {
    "base_dir": TEST_DATA_PATH,
    "filepath": "rabi_test/Rabi_32",
    "custom_output_dir_suffix": "_{fit_backend}_bin_{total_bin}",
    "additional_bins": 4,
       
    "system_name": "Zyla",

    "ROI": "Full", 
    
    "AOI_1_start":  [40, 65],
    "AOI_1_end":    [50,  75], 
    "AOI_2_start":  [20, 100],
    "AOI_2_end":    [30, 110], 
    "single_pixel_check": [90//2, 150//2],
    
    "remove_start_sweep": 1,
    
    "show_tick_marks": True,
    
    "fit_backend": "scipyfit",
    "fit_backend_comparison": ["scipyfit"],
    "fit_pl_pixels": True, 
    "force_fit": True, 
    
    "use_ROI_avg_fit_res_for_all_pixels": True,
    
    "scipyfit_sub_threads": 2, 

    "fit_functions":  {"constant": 1, "damped_rabi": 1},
    
    "normalisation": "div",
    
    "rabi_freq_guess": 10e6,
    "rabi_t_offset_guess": 20e-9, 
    "rabi_amp_guess": 0.0015, 
    "rabi_decay_time_guess": 1e-6,
    
    "rabi_freq_bounds": [1e1, 1e9],
    "rabi_t_offset_bounds": [1e-9, 1e-6],
    "rabi_amp_bounds": [-1, 1],
    "rabi_decay_time_bounds": [1e-9, 1e-4],
    
    "constant_guess": 0.998,
    "constant_bounds": [0, 2],
    
    "save_fig_type": "png",
    
    "colormap_range_dicts": {
        "residual_images": {
            "type": "percentile",
            "values": [2, 98]
        },
        "sigma_images": {
            "type": "percentile",
            "values": [2, 98]
        },
        "param_images": {
            "type": "percentile",
            "values": [10, 90]
        },
    },
    "mpl_rcparams": {
        "legend.loc": "upper right",
    }
}

# if you want to use a reference experiment {ensure you run this even if 'None' as it sets up output dirs etc.}
ref_options_dir = None

In [19]:
options, ref_options = qdmpy.initialize(
    options_dict=options_dict,
    ref_options_dir=ref_options_dir,
    set_mpl_rcparams=True
)

/home/samsc/src/qs/qdmpy_proj/qdmpy_git/src/qdmpy/shared/misc.py:17: QDMPYWarning: Continuing without reference. (No reference chosen or exp_referece_type was 'None')
  warnings.warn(msg, QDMPYWarning)


### Load data

In [ ]:
image, prelim_sweep_list = qdmpy.pl.load_image_and_sweep(options)
PL_image, PL_image_ROI, sig, ref, sig_norm, single_pixel_pl, sweep_list, ROI = qdmpy.pl.reshape_dataset(
        options, image, prelim_sweep_list
    )

ROI_PL_fig = qdmpy.plot.roi_pl_image(options, PL_image)

AOI_PL_fig = qdmpy.plot.aoi_pl_image(options, PL_image_ROI)

AOI_spectra_fig = qdmpy.plot.aoi_spectra(options, sig, ref, sweep_list)

qdmpy.pl.save_pl_data(options, PL_image, PL_image_ROI)

### Load reference experiment - note, you can fit in this same notebook!

In [ ]:
# ref_fit_params, ref_sigmas = qdmpy.pl.load_ref_exp_pl_fit_results(ref_options)

### Fit AOI, ROI averages, single pixel

In [ ]:
fit_model = qdmpy.pl.define_fit_model(options)
backend_ROI_results_lst = qdmpy.pl.fit_roi_avg_pl(options, sig, ref, sweep_list, fit_model) # if update sig_norm => sig,ref
ROI_fit_fig = qdmpy.plot.roi_avg_fits(options, backend_ROI_results_lst)

fit_result_collection_lst = qdmpy.pl.fit_aois_pl(options, sig, ref, single_pixel_pl, sweep_list, fit_model, backend_ROI_results_lst)
AOI_fit_fig = qdmpy.plot.aoi_spectra_fit(options, sig, ref, sweep_list, fit_result_collection_lst, fit_model)

### Fit pixels

In [ ]:
wanted_roi_result = next(
    filter(
        lambda result: result.fit_backend == options["fit_backend"], backend_ROI_results_lst
    )
) # ROI fit result for chosen fit backend
pixel_fit_params, sigmas = qdmpy.pl.get_pl_fit_result(options, sig_norm, sweep_list, fit_model, wanted_roi_result)

qdmpy.pl.save_pl_fit_results(options, pixel_fit_params)
qdmpy.pl.save_pl_fit_sigmas(options, sigmas)

In [ ]:
ref_fit_params, ref_sigmas = qdmpy.pl.load_ref_exp_pl_fit_results(ref_options)

### Plot (fit) parameters

In [ ]:
options["colormap_range_dicts"]["param_images"] = {
            "type": "percentile",
            "values": [10, 90],
}

c_fig = qdmpy.plot.pl_param_images(options, fit_model, pixel_fit_params, "constant")
amp_fig = qdmpy.plot.pl_param_images(options, fit_model, pixel_fit_params, "rabi_amp")
decay_fig = qdmpy.plot.pl_param_images(options, fit_model, pixel_fit_params, "rabi_decay_time")
toffset_fig = qdmpy.plot.pl_param_images(options, fit_model, pixel_fit_params, "rabi_t_offset")
residual_fig = qdmpy.plot.pl_param_images(options, fit_model, pixel_fit_params, "residual")

In [ ]:
options["colormap_range_dicts"]["param_images"] = {
            "type": "percentile",
            "values": [2, 98],
}
exp_fig = qdmpy.plot.pl_param_images(options, fit_model, pixel_fit_params, "rabi_freq")

In [ ]:
qdmpy.save_options(options)
# import qdmpy.shared.json2dict; print(qdmpy.shared.json2dict.dict_to_json_str(options))

In [ ]:
options["used_fastmodel"]